# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()
df.head(5)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,20000.0,17.19%,499.10,RENT,47000.0,Source Verified,Mar-2019,Issued,n,14.02,...,98.0,12.5,0.0,0.0,75824.0,31546.0,33800.0,21524.0,N,N
1,21225.0,14.74%,502.05,MORTGAGE,225000.0,Not Verified,Mar-2019,Issued,n,16.80,...,100.0,50.0,1.0,0.0,747075.0,209426.0,53500.0,128175.0,N,N
2,5000.0,17.97%,180.69,MORTGAGE,62000.0,Not Verified,Mar-2019,Issued,n,19.82,...,66.7,50.0,0.0,0.0,255738.0,31615.0,9400.0,39938.0,N,N
3,20000.0,8.19%,628.49,MORTGAGE,200000.0,Not Verified,Mar-2019,Issued,n,22.66,...,95.7,22.2,0.0,0.0,448069.0,84744.0,49400.0,105180.0,N,N
4,12000.0,15.57%,289.09,MORTGAGE,49000.0,Source Verified,Mar-2019,Issued,n,13.47,...,86.1,14.3,0.0,0.0,189260.0,106025.0,24400.0,68860.0,N,N


In [5]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

In [6]:
def verifyColumns(df):
    for c in df.columns:
        if df[c].dtypes!='float64':
            print(c)
            print(set(df[c]))
verifyColumns(df)


home_ownership
{'MORTGAGE', 'RENT', 'ANY', 'OWN'}
verification_status
{'Source Verified', 'Not Verified', 'Verified'}
issue_d
{'Mar-2019', 'Feb-2019', 'Jan-2019'}
loan_status
{'high_risk', 'low_risk'}
pymnt_plan
{'n'}
initial_list_status
{'f', 'w'}
next_pymnt_d
{'May-2019', 'Apr-2019'}
application_type
{'Individual', 'Joint App'}
hardship_flag
{'N'}
debt_settlement_flag
{'N'}


In [7]:
# Create the training variables by converting the string values into 
# numerical ones using the get_dummies() method.
colList = ['home_ownership','verification_status', 'issue_d',
            'loan_status',  'pymnt_plan',
            'initial_list_status', 'next_pymnt_d',
            'application_type'   , 'hardship_flag',
            'debt_settlement_flag']

df_binary_encoded = pd.get_dummies(df, columns=colList,prefix_sep='_')
df_binary_encoded.head()
verifyColumns(df_binary_encoded)

home_ownership_ANY
{0, 1}
home_ownership_MORTGAGE
{0, 1}
home_ownership_OWN
{0, 1}
home_ownership_RENT
{0, 1}
verification_status_Not Verified
{0, 1}
verification_status_Source Verified
{0, 1}
verification_status_Verified
{0, 1}
issue_d_Feb-2019
{0, 1}
issue_d_Jan-2019
{0, 1}
issue_d_Mar-2019
{0, 1}
loan_status_high_risk
{0, 1}
loan_status_low_risk
{0, 1}
pymnt_plan_n
{1}
initial_list_status_f
{0, 1}
initial_list_status_w
{0, 1}
next_pymnt_d_Apr-2019
{0, 1}
next_pymnt_d_May-2019
{0, 1}
application_type_Individual
{0, 1}
application_type_Joint App
{0, 1}
hardship_flag_N
{1}
debt_settlement_flag_N
{1}


In [8]:
#conver colums from objetc to int
colListEncoded = ['home_ownership_ANY',
'home_ownership_MORTGAGE',
'home_ownership_OWN',
'home_ownership_RENT',
'verification_status_Not Verified',
'verification_status_Source Verified',
'verification_status_Verified',
'issue_d_Feb-2019',
'issue_d_Jan-2019',
'issue_d_Mar-2019',
'loan_status_high_risk',
'loan_status_low_risk',
'pymnt_plan_n',
'initial_list_status_f',
'initial_list_status_w',
'next_pymnt_d_Apr-2019',
'next_pymnt_d_May-2019',
'application_type_Individual',
'application_type_Joint App',
'hardship_flag_N',
'debt_settlement_flag_N']
df_binary_encoded[colListEncoded].astype('int32').dtypes

home_ownership_ANY                     int32
home_ownership_MORTGAGE                int32
home_ownership_OWN                     int32
home_ownership_RENT                    int32
verification_status_Not Verified       int32
verification_status_Source Verified    int32
verification_status_Verified           int32
issue_d_Feb-2019                       int32
issue_d_Jan-2019                       int32
issue_d_Mar-2019                       int32
loan_status_high_risk                  int32
loan_status_low_risk                   int32
pymnt_plan_n                           int32
initial_list_status_f                  int32
initial_list_status_w                  int32
next_pymnt_d_Apr-2019                  int32
next_pymnt_d_May-2019                  int32
application_type_Individual            int32
application_type_Joint App             int32
hardship_flag_N                        int32
debt_settlement_flag_N                 int32
dtype: object

In [9]:
#Create the target variables.
df_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,loan_status_low_risk,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1


# Split the Data into Training and Testing

In [10]:
# Create our features

X = df_binary_encoded.copy().drop(labels=["loan_status_high_risk","loan_status_low_risk"],axis=1)

# Create our target
y = df_binary_encoded.loc[:, ['loan_status_high_risk']].copy()


In [11]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [12]:
# Check the balance of our target values
print(y['loan_status_high_risk'].value_counts()) # high risk 347
  # low risk 68470


0    68470
1      347
Name: loan_status_high_risk, dtype: int64


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10031990)
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(51612, 95)
(17205, 95)
(51612, 1)
(17205, 1)


# Ensemble Learning

## BalancedRandomForestClassifier

In [14]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report,balanced_accuracy_score
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score


In [15]:
# Resample the training data using the BalancedRandomForestClassifier 
# algorithm with 100 estimators.
from imblearn.ensemble import BalancedRandomForestClassifier
clf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)  



BalancedRandomForestClassifier(random_state=0)

## Evaluate the Model 

In [16]:
# After the data is resampled, calculate the accuracy score of the model, 
# generate a confusion matrix, and then print out the imbalanced classification report.
y_pred = clf.predict(X_test)
acc_score = accuracy_score(y_test, y_pred)
print(acc_score)
balanced_accuracy_score(y_test, y_pred)

0.9164196454519035


0.7654862085086489

In [17]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
print(tn,fp,fn,tp)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df


15710 1402 36 57


,Predicted 0,Predicted 1
Actual 0,15710,1402
Actual 1,36,57


In [18]:
print('First Calculation','\n')
total=tn+fp+fn+tp
print('Total Predictions',total)
print('Correct predictions:',tp+tn, round(((tp+tn)/total)*100,2),'%')
print('Incorrect predictions:',fp+fn,round(((fp+fn)/total)*100,2),'%')
p = tp/(tp+fp)
s = tp/(tp+fn)
print('Precision:',round(p,2))
print('Recall:',round(s,2))


print('Specificity:',round(tn/(tn+fp),2))
print('F1-score::',round((2*p*s)/(p+s),2),'\n')

print('Second Calculation','\n')
print(f"Accuracy: {round(accuracy_score(y_test, y_pred), 2)}") 
print(f"Precision: {round(precision_score(y_test, y_pred), 2)}")
print(f"Recall: {round(recall_score(y_test, y_pred), 2)}")
print(f"F1_score: {round(f1_score(y_test, y_pred), 2)}")


First Calculation 

Total Predictions 17205
Correct predictions: 15767 91.64 %
Incorrect predictions: 1438 8.36 %
Precision: 0.04
Recall: 0.61
Specificity: 0.92
F1-score:: 0.07 

Second Calculation 

Accuracy: 0.92
Precision: 0.04
Recall: 0.61
F1_score: 0.07


In [19]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.92      0.61      0.96      0.75      0.58     17112
          1       0.04      0.61      0.92      0.07      0.75      0.55        93

avg / total       0.99      0.92      0.61      0.95      0.75      0.58     17205



In [20]:
#Print the feature importance sorted in descending order 
#(from most to least important feature), along with the feature score.
clf.feature_importances_
sorted(zip(clf.feature_importances_, X.columns), reverse=True)

[(0.07763834505023157, 'total_rec_prncp'),
 (0.06675097268235347, 'total_pymnt_inv'),
 (0.05907311471909396, 'last_pymnt_amnt'),
 (0.051205753054883854, 'total_rec_int'),
 (0.04590413508271411, 'total_pymnt'),
 (0.024629104485206135, 'int_rate'),
 (0.020565445056565147, 'issue_d_Jan-2019'),
 (0.018026586405693767, 'dti'),
 (0.017204042536530417, 'revol_bal'),
 (0.017132801971837374, 'mo_sin_old_rev_tl_op'),
 (0.01666320332469569, 'max_bal_bc'),
 (0.016434216571550368, 'annual_inc'),
 (0.016317341277369096, 'out_prncp_inv'),
 (0.01604740076023514, 'issue_d_Mar-2019'),
 (0.015896440275503715, 'il_util'),
 (0.015301713271614274, 'all_util'),
 (0.015250707292496318, 'mo_sin_old_il_acct'),
 (0.014914327977990029, 'mths_since_recent_inq'),
 (0.014819857868640724, 'total_il_high_credit_limit'),
 (0.014801511740584215, 'total_bc_limit'),
 (0.014465087895609503, 'tot_hi_cred_lim'),
 (0.014226527988419065, 'avg_cur_bal'),
 (0.013923639349161687, 'tot_cur_bal'),
 (0.013810523330857638, 'total_bal

## EasyEnsembleClassifier

In [21]:
#Next, resample the training data using the EasyEnsembleClassifier 
#algorithm with 100 estimators.
from imblearn.ensemble import EasyEnsembleClassifier 
eec = EasyEnsembleClassifier(n_estimators=100,random_state=42)
eec.fit(X_train, y_train) 

y_pred = eec.predict(X_test)



## Evaluate the Model

In [22]:
#After the data is resampled, calculate the accuracy score
#of the model, generate a confusion matrix, 
#and then print out the imbalanced classification report.
acc_score = accuracy_score(y_test, y_pred)
acc_score
print(acc_score)
balanced_accuracy_score(y_test, y_pred)

0.9533856437082243


0.939136278634876

In [23]:
# Display the confusion matrix

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print(tn,fp,fn,tp)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df



16317 795 7 86


,Predicted 0,Predicted 1
Actual 0,16317,795
Actual 1,7,86


In [24]:
print('First Calculation','\n')
total=tn+fp+fn+tp
print('Total Predictions',total)
print('Correct predictions:',tp+tn, round(((tp+tn)/total)*100,2),'%')
print('Incorrect predictions:',fp+fn,round(((fp+fn)/total)*100,2),'%')
p = tp/(tp+fp)
s = tp/(tp+fn)
print('Precision:',round(p,2))
print('Recall:',round(s,2))


print('Specificity:',round(tn/(tn+fp),2))
print('F1-score::',round((2*p*s)/(p+s),2),'\n')

print('Second Calculation','\n')
print(f"Accuracy: {round(accuracy_score(y_test, y_pred), 2)}") 
print(f"Precision: {round(precision_score(y_test, y_pred), 2)}")
print(f"Recall: {round(recall_score(y_test, y_pred), 2)}")
print(f"F1_score: {round(f1_score(y_test, y_pred), 2)}")


First Calculation 

Total Predictions 17205
Correct predictions: 16403 95.34 %
Incorrect predictions: 802 4.66 %
Precision: 0.1
Recall: 0.92
Specificity: 0.95
F1-score:: 0.18 

Second Calculation 

Accuracy: 0.95
Precision: 0.1
Recall: 0.92
F1_score: 0.18


In [25]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.95      0.92      0.98      0.94      0.88     17112
          1       0.10      0.92      0.95      0.18      0.94      0.88        93

avg / total       0.99      0.95      0.92      0.97      0.94      0.88     17205

